In [1]:
groups = ["Ia", "PQsr", "Dl", "Ck"]
wick = prod(groups)

hole = "Ikl"
particle = "aCD"
v = "PQsr"

"PQsr"

In [7]:
wick

"IaPQsrDlCk"

In [2]:
# Dictionary with letters as keys and which index group they are in. i.e. group['D'] => 3
group = Dict(vcat([[(s, i) for s in g] for (i, g) in enumerate(groups)]...));

In [3]:
function isupper(s)
    return uppercase(s) == s
end
function islower(s)
    return lowercase(s) == s
end

islower (generic function with 1 method)

In [12]:
single_contractions = String[]
for s1 in wick
    for s2 in wick
        if group[s1] < group[s2] # Only look for contraction to group ahead
            if isupper(s1) && islower(s2) # Dagger to no dagger
                if s1 in hole*v && s2 in hole*v
                    #Valid Hole Contraction
                    s = s1*s2
                    push!(single_contractions, s)
                end
            elseif islower(s1) && isupper(s2) # No dagger to dagger
                if s1 in particle*v && s2 in particle*v
                    #Valid Particle Contraction
                    s = s1*s2
                    push!(single_contractions, s)
                end
            end
        end
    end
end

In [13]:
using Combinatorics: combinations
contractions = []
for contr in combinations(single_contractions, 5) #All sets of 5 single contractions
    if length(Set(prod(contr))) == 10 #No repeated indices
        c1 = reduce(|, occursin.(r"[PQsr][Dl]", contr)); #H is connected to the first T1
        c2 = reduce(|, occursin.(r"[PQsr][Ck]", contr)); #H is connected to the second T1
        if c1 && c2
            push!(contractions, contr)
        end
    end
end

In [6]:
open("wicks.txt", "w") do wicks
open("deltas.txt", "w") do delta
    count = 0
    for contr in contractions
        count += 1
        if count % 3 == 0
            linebr = "\\\\\n&+ "
        else 
            linebr = "+ "
        end

        l = Dict()
        for (i, single_con) in enumerate(contr)
            l[single_con[1]] = i
            l[single_con[2]] = i
        end
        w = "\\wick{ \\{ \\c$(l['I']) a_i^\\dagger \\c$(l['a']) a_a \\c$(l['P']) a_p^\\dagger "*
            "\\c$(l['Q']) a_q^\\dagger \\c$(l['s']) a_s \\c$(l['r']) a_r \\c$(l['D']) "*
            "a_d^\\dagger \\c$(l['l']) a_l \\c$(l['C']) a_c^\\dagger \\c$(l['k']) a_k \\} }"
        print(wicks, "$w $linebr")

        d = ""
        for single_contr in contr
            s = lowercase(single_contr)
            d *= "\\delta_{$s} "
        end
        print(delta, "$d $linebr")
    end
end
end